In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pydicom
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops

In [2]:
# Path to the directory containing knee DICOM images
input_dir = './Knee_dataset/series-00000/'
output_dir = './KneeOutput-Notebook/'

# Create the output directory if it does not exist
os.makedirs(output_dir, exist_ok=True)

# Get a list of all files in the input directory
dicom_files = [file for file in os.listdir(input_dir) if file.endswith('.dcm')]

In [3]:

# Process each knee image
for file_name in dicom_files:
    # Load the DICOM image
    dicom_data = pydicom.dcmread(os.path.join(input_dir, file_name))
    knee_img = dicom_data.pixel_array

    # Apply Otsu's thresholding to segment knee
    threshold_value = threshold_otsu(knee_img)
    knee_mask = knee_img > threshold_value

    # Label connected components
    labeled_mask = label(knee_mask)

    # Get properties of connected components
    regions = regionprops(labeled_mask)

    # Find the largest connected component (knee area)
    knee_area = None
    max_area = 0
    for region in regions:
        if region.area > max_area:
            max_area = region.area
            knee_area = region

    # Create a binary mask for the largest connected component (knee)
    knee_mask = np.zeros_like(labeled_mask)
    knee_mask[labeled_mask == knee_area.label] = 1

    # Overlay knee contours on the original image and save
    plt.figure(figsize=(8, 8))
    plt.imshow(knee_img, cmap='gray')
    plt.contour(knee_mask, colors='red', linewidths=2, levels=[0.5])
    plt.axis('off')

    # Save the image with knee contours
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'knee_contours_{file_name.replace(".dcm", ".png")}'))
    plt.close()
